In [59]:
import pandas as pd     

In [60]:
#reading in CSV file
care_df = pd.read_csv("care.csv")
care_df.head()

,_id,Indicator,AACC Region,Aboriginality,Age Group,Gender,Count,Year
0,1,Children in care,East Kimberley,NonAboriginal,0 - 17 Years,All,0,2008
1,2,Children in care,East Kimberley,Aboriginal,0 - 17 Years,All,91,2008
2,3,Children in care,Goldfields-Esperance,NonAboriginal,0 - 17 Years,All,34,2008
3,4,Children in care,Goldfields-Esperance,Aboriginal,0 - 17 Years,All,69,2008
4,5,Children in care,Midwest-Gascoyne,Aboriginal,0 - 17 Years,All,94,2008


In [63]:
#reading in CSV file
assessed_df = pd.read_csv("assessed.csv")
assessed_df.head()

,_id,Indicator,AACC Region,Aboriginality,Age Group,Gender,Count,Year
0,1,Safety and Wellbeing Assessments,East Kimberley,Aboriginal,0 - 17 Years,All,325,2015-16
1,2,Safety and Wellbeing Assessments,East Kimberley,NonAboriginal,0 - 17 Years,All,4,2015-16
2,3,Safety and Wellbeing Assessments,Goldfields-Esperance,Aboriginal,0 - 17 Years,All,193,2015-16
3,4,Safety and Wellbeing Assessments,Goldfields-Esperance,NonAboriginal,0 - 17 Years,All,126,2015-16
4,5,Safety and Wellbeing Assessments,Midwest-Gascoyne,Aboriginal,0 - 17 Years,All,471,2015-16


In [64]:
# Delete age group column (not required)
del care_df['Age Group']
care_df.head()

,_id,Indicator,AACC Region,Aboriginality,Gender,Count,Year
0,1,Children in care,East Kimberley,NonAboriginal,All,0,2008
1,2,Children in care,East Kimberley,Aboriginal,All,91,2008
2,3,Children in care,Goldfields-Esperance,NonAboriginal,All,34,2008
3,4,Children in care,Goldfields-Esperance,Aboriginal,All,69,2008
4,5,Children in care,Midwest-Gascoyne,Aboriginal,All,94,2008


In [65]:
#Delete gender (not relevant)
del care_df['Gender']
care_df.head()

,_id,Indicator,AACC Region,Aboriginality,Count,Year
0,1,Children in care,East Kimberley,NonAboriginal,0,2008
1,2,Children in care,East Kimberley,Aboriginal,91,2008
2,3,Children in care,Goldfields-Esperance,NonAboriginal,34,2008
3,4,Children in care,Goldfields-Esperance,Aboriginal,69,2008
4,5,Children in care,Midwest-Gascoyne,Aboriginal,94,2008


In [66]:
#delete years prior to 2015
care2_df = care_df.loc[care_df['Year']>2014]
care2_df.head()

,_id,Indicator,AACC Region,Aboriginality,Count,Year
126,127,Children in care,East Kimberley,Aboriginal,144,2015
127,128,Children in care,East Kimberley,NonAboriginal,0,2015
128,129,Children in care,Goldfields-Esperance,NonAboriginal,23,2015
129,130,Children in care,Goldfields-Esperance,Aboriginal,135,2015
130,131,Children in care,Midwest-Gascoyne,Aboriginal,142,2015


In [67]:
#Delete years after 2016
care3_df = care2_df.loc[care2_df['Year']<2017]
care3_df

,_id,Indicator,AACC Region,Aboriginality,Count,Year
126,127,Children in care,East Kimberley,Aboriginal,144,2015
127,128,Children in care,East Kimberley,NonAboriginal,0,2015
128,129,Children in care,Goldfields-Esperance,NonAboriginal,23,2015
129,130,Children in care,Goldfields-Esperance,Aboriginal,135,2015
130,131,Children in care,Midwest-Gascoyne,Aboriginal,142,2015
131,132,Children in care,Midwest-Gascoyne,NonAboriginal,56,2015
132,133,Children in care,Perth,NonAboriginal,1691,2015
133,134,Children in care,Perth,Aboriginal,1187,2015
134,135,Children in care,Pilbara,NonAboriginal,7,2015
135,136,Children in care,Pilbara,Aboriginal,204,2015


In [68]:
# Delete age group column (not required)
del assessed_df['Age Group']
assessed_df.head()

,_id,Indicator,AACC Region,Aboriginality,Gender,Count,Year
0,1,Safety and Wellbeing Assessments,East Kimberley,Aboriginal,All,325,2015-16
1,2,Safety and Wellbeing Assessments,East Kimberley,NonAboriginal,All,4,2015-16
2,3,Safety and Wellbeing Assessments,Goldfields-Esperance,Aboriginal,All,193,2015-16
3,4,Safety and Wellbeing Assessments,Goldfields-Esperance,NonAboriginal,All,126,2015-16
4,5,Safety and Wellbeing Assessments,Midwest-Gascoyne,Aboriginal,All,471,2015-16


In [69]:
#Delete gender (not relevant)
del assessed_df['Gender']
assessed_df.head()

,_id,Indicator,AACC Region,Aboriginality,Count,Year
0,1,Safety and Wellbeing Assessments,East Kimberley,Aboriginal,325,2015-16
1,2,Safety and Wellbeing Assessments,East Kimberley,NonAboriginal,4,2015-16
2,3,Safety and Wellbeing Assessments,Goldfields-Esperance,Aboriginal,193,2015-16
3,4,Safety and Wellbeing Assessments,Goldfields-Esperance,NonAboriginal,126,2015-16
4,5,Safety and Wellbeing Assessments,Midwest-Gascoyne,Aboriginal,471,2015-16


In [70]:
#Remove dash in year by replacing with no space
assessed_df['Year'] = assessed_df['Year'].str.replace(r'-','')


In [71]:
assessed_df.head()

,_id,Indicator,AACC Region,Aboriginality,Count,Year
0,1,Safety and Wellbeing Assessments,East Kimberley,Aboriginal,325,201516
1,2,Safety and Wellbeing Assessments,East Kimberley,NonAboriginal,4,201516
2,3,Safety and Wellbeing Assessments,Goldfields-Esperance,Aboriginal,193,201516
3,4,Safety and Wellbeing Assessments,Goldfields-Esperance,NonAboriginal,126,201516
4,5,Safety and Wellbeing Assessments,Midwest-Gascoyne,Aboriginal,471,201516


In [72]:
#Remove last 2 digits to make single year (do not re run as it removes entire year)
assessed_df['Year'] = assessed_df['Year'].str[:-2]

In [73]:
assessed_df.head()

,_id,Indicator,AACC Region,Aboriginality,Count,Year
0,1,Safety and Wellbeing Assessments,East Kimberley,Aboriginal,325,2015
1,2,Safety and Wellbeing Assessments,East Kimberley,NonAboriginal,4,2015
2,3,Safety and Wellbeing Assessments,Goldfields-Esperance,Aboriginal,193,2015
3,4,Safety and Wellbeing Assessments,Goldfields-Esperance,NonAboriginal,126,2015
4,5,Safety and Wellbeing Assessments,Midwest-Gascoyne,Aboriginal,471,2015


In [74]:
assessed_df['AACC Region'].unique()


array(['East Kimberley', 'Goldfields-Esperance', 'Midwest-Gascoyne',
       'Perth', 'Pilbara', 'Southern', 'Western Australia', 'Wheatbelt',
       'West Kimberley'], dtype=object)

In [75]:
care3_df['AACC Region'].unique()

array(['East Kimberley', 'Goldfields-Esperance', 'Midwest-Gascoyne',
       'Perth', 'Pilbara', 'Southern', 'Western Australia', 'Wheatbelt',
       'West Kimberley'], dtype=object)